**Set environment**

In [3]:
import sys
sys.path.append('../')
from config_sing import *

You are on Duke Server: Singularity: Proj CombEffect
BASE DIRECTORY:     /mount/work
PATH OF SOURCE:     /mount/work/source
PATH OF EXECUTABLE: /mount/work/exe
PATH OF ANNOTATION: /mount/work/annotation
PATH OF PROJECT:    /mount/project
PATH OF RESULTS:    /mount/work/out/proj_combeffect



In [4]:
import sqlite3
import itertools as it

## Test: check where the data is

In [3]:
fdiry = os.path.join(FD_RES, "annotation_fragment")
os.listdir(fdiry)

['TFX_DMSO',
 'Input5_20x',
 'Input1_20x',
 'TFX2_DMSO',
 'TFX_Dex',
 'Input3',
 '.ipynb_checkpoints',
 'TFX5_Dex',
 'Input4_20x',
 'filter_motif_score095',
 'target_PER1_MA0099.3.bed',
 'TFX2_Dex',
 'TFX5_DMSO',
 'TFX4_Dex',
 'Input4',
 'merge',
 'Input5',
 'target_PER1_MA0113.3.bed',
 'Input1',
 'Input2',
 'Input',
 'TFX3_Dex',
 'TFX4_DMSO',
 'Input2_20x',
 'TFX3_DMSO',
 'Input3_20x']

In [4]:
###
fdirys = list()

###
samples = "Input?,Input?_20x,TFX?_DMSO,TFX?_Dex".split(",")
for sam in samples:
    fglob = os.path.join(FD_RES, "annotation_fragment", sam)
    lst   = glob.glob(fglob)
    lst.sort()
    fdirys += lst

###
for fdiry in fdirys:
    print(fdiry)

/mount/work/out/proj_combeffect/annotation_fragment/Input1
/mount/work/out/proj_combeffect/annotation_fragment/Input2
/mount/work/out/proj_combeffect/annotation_fragment/Input3
/mount/work/out/proj_combeffect/annotation_fragment/Input4
/mount/work/out/proj_combeffect/annotation_fragment/Input5
/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x
/mount/work/out/proj_combeffect/annotation_fragment/Input2_20x
/mount/work/out/proj_combeffect/annotation_fragment/Input3_20x
/mount/work/out/proj_combeffect/annotation_fragment/Input4_20x
/mount/work/out/proj_combeffect/annotation_fragment/Input5_20x
/mount/work/out/proj_combeffect/annotation_fragment/TFX2_DMSO
/mount/work/out/proj_combeffect/annotation_fragment/TFX3_DMSO
/mount/work/out/proj_combeffect/annotation_fragment/TFX4_DMSO
/mount/work/out/proj_combeffect/annotation_fragment/TFX5_DMSO
/mount/work/out/proj_combeffect/annotation_fragment/TFX2_Dex
/mount/work/out/proj_combeffect/annotation_fragment/TFX3_Dex
/mount/work/out/proj

In [5]:
%%bash
FPATH="/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1.bed.gz"
zcat ${FPATH} | head

chr17	8148117	8149012	1	chr17	8148107	8148124	KLF/SP/2	9.0318	7
chr17	8148117	8149012	1	chr17	8148109	8148121	INSM1	6.1647	4
chr17	8148117	8149012	1	chr17	8148109	8148129	GC-tract	8.3277	12
chr17	8148117	8149012	1	chr17	8148123	8148133	GLI	7.4318	10
chr17	8148117	8149012	1	chr17	8148124	8148139	NR/17	7.8649	15
chr17	8148117	8149012	1	chr17	8148126	8148137	KLF/SP/1	11.3678	11
chr17	8148117	8149012	1	chr17	8148129	8148147	REST/NRSF	4.8708	18
chr17	8148117	8149012	1	chr17	8148132	8148142	CREB/ATF/2	7.1033	10
chr17	8148117	8149012	1	chr17	8148132	8148142	CREB/ATF/3	7.017	10
chr17	8148117	8149012	1	chr17	8148133	8148142	CREB/ATF/1	7.7524	9


## Test: read in the data

In [6]:
###
fdirys = list()
samples = "Input?,Input?_20x,TFX?_DMSO,TFX?_Dex".split(",")
for sam in samples:
    fglob = os.path.join(FD_RES, "annotation_fragment", sam)
    lst   = glob.glob(fglob)
    lst.sort()
    fdirys += lst

###
fname = "target_PER1.bed.gz"
for fdiry in fdirys:
    fpath = os.path.join(fdiry, fname)
    print(fpath)

/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input5/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input2_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input3_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input4_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/Input5_20x/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/TFX2_DMSO/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annotation_fragment/TFX3_DMSO/target_PER1.bed.gz
/mount/work/out/proj_combeffect/annota

In [7]:
fdiry = fdirys[0]
fname = "target_PER1.bed.gz"
fpath = os.path.join(fdiry, fname)
print(fpath)

n_lines = 5
with gzip.open(fpath, "rb") as file:
    lines  = it.islice(file, n_lines)
    for line in lines:
        lst = line.decode('ASCII').strip().split('\t')  
        print(lst)

/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1.bed.gz
['chr17', '8148117', '8149012', '1', 'chr17', '8148107', '8148124', 'KLF/SP/2', '9.0318', '7']
['chr17', '8148117', '8149012', '1', 'chr17', '8148109', '8148121', 'INSM1', '6.1647', '4']
['chr17', '8148117', '8149012', '1', 'chr17', '8148109', '8148129', 'GC-tract', '8.3277', '12']
['chr17', '8148117', '8149012', '1', 'chr17', '8148123', '8148133', 'GLI', '7.4318', '10']
['chr17', '8148117', '8149012', '1', 'chr17', '8148124', '8148139', 'NR/17', '7.8649', '15']


In [11]:
def prep_line(lst):
    ### parse info and get fragment and motif ID
    fragment = "_".join(lst[:3])
    binding  = "_".join(lst[4:8])
    overlap  = int(lst[-1])
    
    ### calc motif lengths
    mtf_len  = int(lst[6]) - int(lst[5])
    
    return mtf_len == overlap, (fragment, binding)

In [12]:
fdiry = fdirys[0]
fname = "target_PER1.bed.gz"
fpath = os.path.join(fdiry, fname)

n_lines = 5
with gzip.open(fpath, "rb") as file:
    lines  = it.islice(file, n_lines)
    for line in lines:
        lst = line.decode('ASCII').strip().split('\t')
        is_cover, row = prep_line(lst)
        print(lst)
        print(is_cover, row)
        if is_cover:
            print(row)
        print()

['chr17', '8148117', '8149012', '1', 'chr17', '8148107', '8148124', 'KLF/SP/2', '9.0318', '7']
False ('chr17_8148117_8149012', 'chr17_8148107_8148124_KLF/SP/2')

['chr17', '8148117', '8149012', '1', 'chr17', '8148109', '8148121', 'INSM1', '6.1647', '4']
False ('chr17_8148117_8149012', 'chr17_8148109_8148121_INSM1')

['chr17', '8148117', '8149012', '1', 'chr17', '8148109', '8148129', 'GC-tract', '8.3277', '12']
False ('chr17_8148117_8149012', 'chr17_8148109_8148129_GC-tract')

['chr17', '8148117', '8149012', '1', 'chr17', '8148123', '8148133', 'GLI', '7.4318', '10']
True ('chr17_8148117_8149012', 'chr17_8148123_8148133_GLI')
('chr17_8148117_8149012', 'chr17_8148123_8148133_GLI')

['chr17', '8148117', '8149012', '1', 'chr17', '8148124', '8148139', 'NR/17', '7.8649', '15']
True ('chr17_8148117_8149012', 'chr17_8148124_8148139_NR/17')
('chr17_8148117_8149012', 'chr17_8148124_8148139_NR/17')



## Test: Generate table

```
- building the related tables
    - Annotation
    - Count
    - InputScore
```

**Query**

In [13]:
query_reset = ("DROP TABLE IF EXISTS Annotation")

query_table = ("""
    CREATE TABLE IF NOT EXISTS Annotation (
        fragment TEXT, 
        binding  TEXT,
        FOREIGN KEY (fragment) REFERENCES Fragment (fragment),
        FOREIGN KEY (binding)  REFERENCES Motif    (binding),
        UNIQUE (fragment, binding) ON CONFLICT IGNORE
    );""")

query_insert = ("""
    INSERT OR IGNORE INTO Annotation
        (fragment, binding)
    VALUES 
        (?,?)
    """)

**Insert**

In [14]:
def prep_line(lst):
    ### parse info and get fragment and motif ID
    fragment = "_".join(lst[:3])
    binding  = "_".join(lst[4:8])
    overlap  = int(lst[-1])
    
    ### calc motif lengths
    mtf_len  = int(lst[6]) - int(lst[5])
    
    return mtf_len == overlap, (fragment, binding)

In [18]:
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

fdiry = fdirys[0]
fname = "target_PER1.bed.gz"
fpath_gz = os.path.join(fdiry, fname)


with sqlite3.connect(fpath_db) as conn, gzip.open(fpath_gz, "rb") as file:
    ### reset
    cursor = conn.cursor()
    query  = query_reset
    cursor.execute(query)
    
    ### create table
    cursor = conn.cursor()
    query  = query_table
    cursor.execute(query)
    
    ### insert values
    query  = query_insert
    
    n_lines = 5
    with gzip.open(fpath_gz, "rb") as file:
        lines  = it.islice(file, n_lines)
        for line in lines:
            lst = line.decode('ASCII').strip().split('\t')
            is_cover, row = prep_line(lst)
            if is_cover:
                cursor.execute(query, row)
    
        ### show that the table is created
        cursor.execute("SELECT * FROM Annotation")
        for row in cursor.fetchall():
            print(row)

('chr17_8148117_8149012', 'chr17_8148123_8148133_GLI')
('chr17_8148117_8149012', 'chr17_8148124_8148139_NR/17')


## Insert the whole file

In [19]:
###
fdirys = list()
samples = "Input?,Input?_20x,TFX?_DMSO,TFX?_Dex".split(",")
for sam in samples:
    fglob = os.path.join(FD_RES, "annotation_fragment", sam)
    lst   = glob.glob(fglob)
    lst.sort()
    fdirys += lst

###
fname = "target_PER1.bed.gz"
fpaths_gz = [os.path.join(fdiry, fname) for fdiry in fdirys]
fpaths_gz

['/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input5/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input2_20x/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input3_20x/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input4_20x/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/Input5_20x/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/TFX2_DMSO/target_PER1.bed.gz',
 '/mount/work/out/proj_combeffect/annotation_fragment/TFX3_DMSO/target_PER1

In [20]:
def prep_line(lst):
    ### parse info and get fragment and motif ID
    fragment = "_".join(lst[:3])
    binding  = "_".join(lst[4:8])
    overlap  = int(lst[-1])
    
    ### calc motif lengths
    mtf_len  = int(lst[6]) - int(lst[5])
    
    return mtf_len == overlap, (fragment, binding)

In [21]:
%%time

fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

counter_tot = 0
with sqlite3.connect(fpath_db) as conn:
    ### reset
    cursor = conn.cursor()
    query  = query_reset
    cursor.execute(query)
    
   ### loop through each annotation file
    for fpath_gz in fpaths_gz:
        ### init, show progress
        print(fpath_gz, flush=True)
        counter = 0
        
        with gzip.open(fpath_gz, "rb") as file:
            ### create table if not exist
            cursor = conn.cursor()
            query  = query_table
            cursor.execute(query)

            ### insert values
            query  = query_insert
            lines  = file #it.islice(file, n_lines)
            for line in lines:
                lst = line.decode('ASCII').strip().split('\t')  
                is_cover, row = prep_line(lst)
                counter_tot += is_cover
                counter     += is_cover
                if is_cover:
                    cursor.execute(query, row)
        print("#Rows Inserted:", counter, flush=True)
        
print()            
print("#Rows Total:", counter_tot)

/mount/work/out/proj_combeffect/annotation_fragment/Input1/target_PER1.bed.gz
#Rows Inserted: 37492
/mount/work/out/proj_combeffect/annotation_fragment/Input2/target_PER1.bed.gz
#Rows Inserted: 44343
/mount/work/out/proj_combeffect/annotation_fragment/Input3/target_PER1.bed.gz
#Rows Inserted: 57856
/mount/work/out/proj_combeffect/annotation_fragment/Input4/target_PER1.bed.gz
#Rows Inserted: 39747
/mount/work/out/proj_combeffect/annotation_fragment/Input5/target_PER1.bed.gz
#Rows Inserted: 28704
/mount/work/out/proj_combeffect/annotation_fragment/Input1_20x/target_PER1.bed.gz
#Rows Inserted: 704118
/mount/work/out/proj_combeffect/annotation_fragment/Input2_20x/target_PER1.bed.gz
#Rows Inserted: 685965
/mount/work/out/proj_combeffect/annotation_fragment/Input3_20x/target_PER1.bed.gz
#Rows Inserted: 693566
/mount/work/out/proj_combeffect/annotation_fragment/Input4_20x/target_PER1.bed.gz
#Rows Inserted: 766079
/mount/work/out/proj_combeffect/annotation_fragment/Input5_20x/target_PER1.bed.g

**Check**

In [22]:
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = "select count(*) from Annotation"
    cursor.execute(query)
    print(cursor.fetchall())

[(3487257,)]


## Create index

In [5]:
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = "select count(*) from Annotation"
    cursor.execute(query)
    print(cursor.fetchall())

[(3487257,)]


In [7]:
query_out = """
    SELECT name 
    FROM sqlite_master 
    WHERE type = 'index';
    """
###
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    query  = query_out
    cursor = conn.cursor()
    cursor.execute(query)
    rows   = cursor.fetchall()
rows

[('sqlite_autoindex_Sample_1',),
 ('sqlite_autoindex_Fragment_1',),
 ('sqlite_autoindex_Motif_1',),
 ('idx_location',),
 ('idx_motif_loc',),
 ('sqlite_autoindex_Annotation_1',)]

In [8]:
query_index  = """CREATE INDEX idx_annot_frag ON Annotation (fragment)"""

In [9]:
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = query_index
    cursor.execute(query)

In [10]:
query_out = """
    SELECT name 
    FROM sqlite_master 
    WHERE type = 'index';
    """
###
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    query  = query_out
    cursor = conn.cursor()
    cursor.execute(query)
    rows   = cursor.fetchall()
rows

[('sqlite_autoindex_Sample_1',),
 ('sqlite_autoindex_Fragment_1',),
 ('sqlite_autoindex_Motif_1',),
 ('idx_location',),
 ('idx_motif_loc',),
 ('sqlite_autoindex_Annotation_1',),
 ('idx_annot_frag',)]